# Get Author info from papers
https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data/operation/get_graph_get_paper_authors
https://api.semanticscholar.org/api-docs/graph#tag/Author-Data/operation/post_graph_get_authors

In [1]:
import os
import requests
import json
import pandas as pd
import publication_query

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

In [2]:
KG_PATH = "../kg/data"

In [3]:
ALPHA = 0.5
BETA = 0.5

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/christian-horgan/pubmed-search/main/data/article_data_comparison.csv")

In [5]:
df.head()

,pm_id,title,pmc_id,authors,doi,keywords,mesh_ids,mesh_terms,abstract,project_serial_num,journal_name,publication_year
0,37727605,Repurposing a SARS-CoV-2 surveillance program ...,PMC10505707,"King KL,Ham R,Smothers A,Lee I,Bowie T,Teetsel...",doi:10.3389/fpubh.2023.1168551,"SARS-CoV-2,flu,influenza,saliva,surveillance t...","D006801,D000086402,D014495,D000086382,D007251,...","Humans,SARS-CoV-2,Universities,COVID-19,Influe...",Standard multiplex RT-qPCR diagnostic tests us...,AA029328,Frontiers in public health,2023
1,37590297,A capacitive laser-induced graphene based apta...,PMC10434860,"Moreira G,Qian H,Datta SPA,Bliznyuk N,Carpente...",doi:10.1371/journal.pone.0290256,NaN,"D006801,D000086402,D000086382,D006108,D007251,...","Humans,SARS-CoV-2,COVID-19,Graphite,Influenza,...",SARS-CoV-2 virus induced CoVID-19 pandemic has...,AA029328,PloS one,2023
2,36595104,Ion-selective electrodes based on laser-induce...,NaN,"Soares RRA,Hjort RG,Pola CC,Jing D,Cecon VS,Cl...",doi:10.1007/s00604-022-05615-9,"Electrochemical Sensors,Food additives,Food sa...","D006801,D017736,D006108,D009573,D014867,D007834","Humans,Ion-Selective Electrodes,Graphite,Nitri...",Nitrite is an important food additive for cure...,AA029328,Mikrochimica acta,2023
3,36354449,An Experimental Framework for Developing Point...,PMC9688365,"Ullah SF,Moreira G,Datta SPA,McLamore E,Vanegas D",doi:10.3390/bios12110938,"SARS-CoV-2,analytical sensing,binding kinetics...","D006801,D058266,D000086402,D000086382,D007368,...","Humans,Dielectric Spectroscopy,SARS-CoV-2,COVI...",Biolayer interferometry (BLI) is a well-establ...,AA029328,Biosensors,2022
4,35992634,Development of a Biosensor Based on Angiotensi...,PMC9386735,"Moreira G,Casso-Hartmann L,Datta SPA,Dean D,Mc...",doi:10.3389/fsens.2022.917380,"LIG electrodes,attenuated virus,betacoronaviru...",NaN,NaN,Severe acute respiratory syndrome coronavirus ...,AA029328,Frontiers in sensors,2022


In [6]:
print(df.shape[0])
df.drop_duplicates(inplace=True)
print(df.shape[0])

393
393


In [7]:
print(df["pm_id"].nunique())
print(df["doi"].nunique())
print(df["title"].nunique())

380
376
377


In [8]:
df["pmId"] = df["pm_id"].astype(str)
df["pmidCurie"] = "pmid:" + df["pmId"]
paper_ids = list(df["pmidCurie"].unique())
print("paper_ids:", len(paper_ids))

paper_ids: 380


In [9]:
df.head()

,pm_id,title,pmc_id,authors,doi,keywords,mesh_ids,mesh_terms,abstract,project_serial_num,journal_name,publication_year,pmId,pmidCurie
0,37727605,Repurposing a SARS-CoV-2 surveillance program ...,PMC10505707,"King KL,Ham R,Smothers A,Lee I,Bowie T,Teetsel...",doi:10.3389/fpubh.2023.1168551,"SARS-CoV-2,flu,influenza,saliva,surveillance t...","D006801,D000086402,D014495,D000086382,D007251,...","Humans,SARS-CoV-2,Universities,COVID-19,Influe...",Standard multiplex RT-qPCR diagnostic tests us...,AA029328,Frontiers in public health,2023,37727605,pmid:37727605
1,37590297,A capacitive laser-induced graphene based apta...,PMC10434860,"Moreira G,Qian H,Datta SPA,Bliznyuk N,Carpente...",doi:10.1371/journal.pone.0290256,NaN,"D006801,D000086402,D000086382,D006108,D007251,...","Humans,SARS-CoV-2,COVID-19,Graphite,Influenza,...",SARS-CoV-2 virus induced CoVID-19 pandemic has...,AA029328,PloS one,2023,37590297,pmid:37590297
2,36595104,Ion-selective electrodes based on laser-induce...,NaN,"Soares RRA,Hjort RG,Pola CC,Jing D,Cecon VS,Cl...",doi:10.1007/s00604-022-05615-9,"Electrochemical Sensors,Food additives,Food sa...","D006801,D017736,D006108,D009573,D014867,D007834","Humans,Ion-Selective Electrodes,Graphite,Nitri...",Nitrite is an important food additive for cure...,AA029328,Mikrochimica acta,2023,36595104,pmid:36595104
3,36354449,An Experimental Framework for Developing Point...,PMC9688365,"Ullah SF,Moreira G,Datta SPA,McLamore E,Vanegas D",doi:10.3390/bios12110938,"SARS-CoV-2,analytical sensing,binding kinetics...","D006801,D058266,D000086402,D000086382,D007368,...","Humans,Dielectric Spectroscopy,SARS-CoV-2,COVI...",Biolayer interferometry (BLI) is a well-establ...,AA029328,Biosensors,2022,36354449,pmid:36354449
4,35992634,Development of a Biosensor Based on Angiotensi...,PMC9386735,"Moreira G,Casso-Hartmann L,Datta SPA,Dean D,Mc...",doi:10.3389/fsens.2022.917380,"LIG electrodes,attenuated virus,betacoronaviru...",NaN,NaN,Severe acute respiratory syndrome coronavirus ...,AA029328,Frontiers in sensors,2022,35992634,pmid:35992634


In [10]:
#paper_ids = ["pmid:35992634", "pmid:37683185", "doi:10.1007/s00604-022-05615-9", "doi:10.1101/2023.06.05.23290966", "doi:10.1101/2023.02.27.530294", "URL:https://www.biorxiv.org/content/10.1101/2023.02.27.530294v1", "doi:10.1101/2023.05.04.539416"] # 4: medRxiv, #5 bioRxiv, 6 bioRxiv URL, 7 bioRxiv not found

In [11]:
# DOIs from Semantic Scholar sometimes use the DOI from a prepint even though the the paper has been published in a per-reviewed journal and the new DOI is available.

In [12]:
authors = publication_query.get_author_ids(paper_ids)

Number of mismatches: 9


In [13]:
# merge by both doi and pmId (both give partial matches) and then combine the results.
authors_wo_pmid = authors.drop(columns="pmId")
authors_wo_pmid.drop_duplicates(inplace=True)
authors_doi = df.merge(authors_wo_pmid, on="doi").copy()

authors_wo_doi = authors.drop(columns="doi")
authors_wo_doi.drop_duplicates(inplace=True)
authors_pmid= df.merge(authors_wo_doi, on="pmId").copy()

authors_primary = pd.concat([authors_doi, authors_pmid])
authors_primary.drop_duplicates(inplace=True)

In [14]:
authors_primary.head()

,pm_id,title,pmc_id,authors,doi,keywords,mesh_ids,mesh_terms,abstract,project_serial_num,journal_name,publication_year,pmId,pmidCurie,authorId,name,paperCount,citationCount,hIndex,paperId,pmcId
0,37727605,Repurposing a SARS-CoV-2 surveillance program ...,PMC10505707,"King KL,Ham R,Smothers A,Lee I,Bowie T,Teetsel...",doi:10.3389/fpubh.2023.1168551,"SARS-CoV-2,flu,influenza,saliva,surveillance t...","D006801,D000086402,D014495,D000086382,D007251,...","Humans,SARS-CoV-2,Universities,COVID-19,Influe...",Standard multiplex RT-qPCR diagnostic tests us...,AA029328,Frontiers in public health,2023,37727605,pmid:37727605,2154342066,Kylie L King,5,41,4,8a26272b10779c0d0cedc6eb4e2630d906429658,10505707
1,37727605,Repurposing a SARS-CoV-2 surveillance program ...,PMC10505707,"King KL,Ham R,Smothers A,Lee I,Bowie T,Teetsel...",doi:10.3389/fpubh.2023.1168551,"SARS-CoV-2,flu,influenza,saliva,surveillance t...","D006801,D000086402,D014495,D000086382,D007251,...","Humans,SARS-CoV-2,Universities,COVID-19,Influe...",Standard multiplex RT-qPCR diagnostic tests us...,AA029328,Frontiers in public health,2023,37727605,pmid:37727605,2043300888,Rachel E. Ham,6,22,3,8a26272b10779c0d0cedc6eb4e2630d906429658,10505707
2,37727605,Repurposing a SARS-CoV-2 surveillance program ...,PMC10505707,"King KL,Ham R,Smothers A,Lee I,Bowie T,Teetsel...",doi:10.3389/fpubh.2023.1168551,"SARS-CoV-2,flu,influenza,saliva,surveillance t...","D006801,D000086402,D014495,D000086382,D007251,...","Humans,SARS-CoV-2,Universities,COVID-19,Influe...",Standard multiplex RT-qPCR diagnostic tests us...,AA029328,Frontiers in public health,2023,37727605,pmid:37727605,2154336577,Austin R. Smothers,6,17,2,8a26272b10779c0d0cedc6eb4e2630d906429658,10505707
3,37727605,Repurposing a SARS-CoV-2 surveillance program ...,PMC10505707,"King KL,Ham R,Smothers A,Lee I,Bowie T,Teetsel...",doi:10.3389/fpubh.2023.1168551,"SARS-CoV-2,flu,influenza,saliva,surveillance t...","D006801,D000086402,D014495,D000086382,D007251,...","Humans,SARS-CoV-2,Universities,COVID-19,Influe...",Standard multiplex RT-qPCR diagnostic tests us...,AA029328,Frontiers in public health,2023,37727605,pmid:37727605,2237391763,Isaac Lee,1,0,0,8a26272b10779c0d0cedc6eb4e2630d906429658,10505707
4,37727605,Repurposing a SARS-CoV-2 surveillance program ...,PMC10505707,"King KL,Ham R,Smothers A,Lee I,Bowie T,Teetsel...",doi:10.3389/fpubh.2023.1168551,"SARS-CoV-2,flu,influenza,saliva,surveillance t...","D006801,D000086402,D014495,D000086382,D007251,...","Humans,SARS-CoV-2,Universities,COVID-19,Influe...",Standard multiplex RT-qPCR diagnostic tests us...,AA029328,Frontiers in public health,2023,37727605,pmid:37727605,2237321471,Tyler Bowie,1,0,0,8a26272b10779c0d0cedc6eb4e2630d906429658,10505707


In [15]:
# rename and reorder columns
#col_map = {"authorId": "id", "name": "name", "paperCount": "paperCount", "citationCount": "citationCount"}
#researchers_primary = publication_query.rename_and_reorder_columns(authors_primary, col_map)
researchers_primary = authors_primary[["authorId", "name", "paperCount", "citationCount"]].copy()
researchers_primary.drop_duplicates(inplace=True)

In [16]:
researchers_primary.head()

,authorId,name,paperCount,citationCount
0,2154342066,Kylie L King,5,41
1,2043300888,Rachel E. Ham,6,22
2,2154336577,Austin R. Smothers,6,17
3,2237391763,Isaac Lee,1,0
4,2237321471,Tyler Bowie,1,0


In [17]:
def create_name_cols(full_name):
    """ Split full name into parts. """
    parts = full_name.split(" ")
    first_name = parts[0].replace(".", "")
    last_name = parts[-1]
    middle_name = ""
    middle_initials = ""
    n_middle_initials = len(parts) - 2
    if n_middle_initials > 0:
        for part in parts[1: n_middle_initials+1]:
            middle_name += part
            middle_initials += part[:1]
    middle_name = middle_name.replace(" ", "")
    middle_name = middle_name.replace(".", "")
    # name = last name + initials
    name = last_name + " " + first_name[:1] + middle_initials

    return name, full_name, first_name, middle_name, last_name

researchers_primary[['name', 'fullName', 'firstName', 'middleName', 'lastName']] = researchers_primary['name'].apply(lambda x: pd.Series(create_name_cols(x)))

In [20]:
researchers_primary.drop_duplicates(inplace=True)
researchers_primary.dropna(inplace=True)
researchers_primary.head()

,id,name,fullName,firstName,middleName,lastName,paperCount,citationCount
0,2154342066,King KL,Kylie L King,Kylie,L,King,5,41
1,2043300888,Ham RE,Rachel E. Ham,Rachel,E,Ham,6,22
2,2154336577,Smothers AR,Austin R. Smothers,Austin,R,Smothers,6,17
3,2237391763,Lee I,Isaac Lee,Isaac,,Lee,1,0
4,2237321471,Bowie T,Tyler Bowie,Tyler,,Bowie,1,0


In [21]:
researchers_primary.to_csv(os.path.join(KG_PATH, "nodes", "Researcher_primary_authors.csv"), index=False)

In [ ]:
matched_paper_ids = list(authors_primary["paperId"].unique())
citations = publication_query.get_citations(matched_paper_ids)
citations.head()

In [ ]:
pub = publication_query.get_publication_info(matched_paper_ids)
# set publication type as primary citation
pub["type"] = "primary"
distance = 1 # for primary citation
pub = publication_query.add_relevance_score(pub, ["title", "abstract"], distance)
pub["relevance"].value_counts()

In [ ]:
# embeddings = publication_query.get_embeddings(matched_paper_ids)
# embeddings.head()

In [ ]:
pub.head(100)

In [ ]:
cited_papers = list(citations["citedPaperId"].unique())

In [ ]:
len(cited_papers)

In [ ]:
# some papers have no pmId, not in PubMed
cit = publication_query.get_publication_info(cited_papers)
# set publication type as secondary citation
cit["type"] = "secondary"
distance = 2 # for secondary citation
cit = publication_query.add_relevance_score(cit, ["title", "abstract"], distance)

In [ ]:
cit["relevance"].value_counts()

In [ ]:
cit.head()

In [ ]:
authors_secondary = publication_query.get_author_ids(cited_papers)
authors_secondary.head()

In [ ]:
# import json
# r = requests.post(
#     'https://api.semanticscholar.org/graph/v1/author/batch',
#     params={'fields': 'name,homepage,hIndex,citationCount'},
#     json={"ids":["1741101", "1780531"]}
# )
# print(json.dumps(r.json(), indent=2))

In [ ]:
# def get_embbedings(paper_ids):
#     data = get_paper_data(paper_ids, "citations.paperId,citationCount")
#     #print(json.dumps(data, indent=2))
#     df = pd.json_normalize(data, record_path=["citations"], record_prefix="citation.", meta=["paperId", "citationCount"])
#     df.rename(columns={"citation.paperId": "citedPaperId"}, inplace=True)
#     return df[["paperId", "citationCount", "citedPaperId"]]